In [ ]:
import hydrobr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
def municipios(municipios, estado):
  if len(municipios) > 0:
    tabela = []
    for i in range(0, len(municipios)):
      estacao = hydrobr.get_data.ANA.list_prec_stations(estado = estado, city = municipios[i])
      tabela.append(estacao)
    tabela_final = pd.concat(tabela)
    display(tabela_final)
    return tabela_final
  else:
    estacao = hydrobr.get_data.ANA.list_prec_stations(estado = estado)
    display(estacao)
    return estacao

In [ ]:
def data_gantt(tabela, codigo = 0):
  if codigo == 0:
    estacoes = tabela.Code.to_list()
    dados = hydrobr.get_data.ANA.prec_data(estacoes)
    id = tabela["Código"]
    name = tabela["Nome"]
    responsible = tabela["Responsável"]
    id_lista = []
    nome_lista = []
    responsavel_lista = []
    for i in range(0, len(tabela)):
      id_lista.append(id.iloc[i])
    for i in range(0, len(tabela)):
      nome_lista.append(name.iloc[i])
    for i in range(0, len(tabela)):
      responsavel_lista.append(responsible.iloc[i])
    for i in range(0, len(id_lista)):
      dados = dados.rename(columns={id_lista[i]: nome_lista[i] + " " + "(" + responsavel_lista[i] + ")"})
    return dados
  if codigo == 1:
      estacoes = tabela.Code.to_list()
      dados = hydrobr.get_data.ANA.prec_data(estacoes)
      return dados

In [ ]:
municipios_lista = ['MUCURI', 'RODELAS'] # Escolha os municípios que deseja coletar. Deixe a lista vazia caso queira coletar todos do estado escolhido abaixo
estado = 'BAHIA'                         # Escolha o estado ao qual os municípios acima pertecem. Escolha apenas um
tabela = municipios(municipios_lista, estado)

In [ ]:
estacao = gpd.GeoDataFrame(tabela, geometry = gpd.points_from_xy(tabela['Longitude'], tabela['Latitude']))
raster = rasterio.open('/content/gdrive/MyDrive/TIFS/2021-02-09.tif') # Substitua com o caminho e nome da sua imagem TIF
dados = data_gantt(tabela, 1) # 0 mostrará o nome das estações no diagrama Gantt, enquanto 1 mostrará seus códigos

In [ ]:
img, axis = plt.subplots(figsize = (10,8))

show(raster, ax = axis)
estacao.plot(ax = axis, color = 'red', alpha = 0.4)

In [ ]:
gantt = hydrobr.Plot.gantt(data)

gantt.update_layout(
    autosize=True,
    width=1300,
    height=500,
    xaxis_title = 'Ano',
    yaxis_title = 'Estações',
    font=dict(family="Courier New", size=15))

display(gantt)